# Example on how to use E-NAUTILUS with a pre-computed Pareto front
We begin by importing the necessary libraries.

In [ ]:
import logging

import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler  # for scaling the data
import ipywidgets as widgets
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from desdeo.problem.Problem import ScalarDataProblem
from desdeo.methods.Nautilus import ENautilus

logging.getLogger("matplotlib").setLevel(logging.WARNING)

For later use, define a function to display the objective functions as polar scatter plots and a function to create a widget to select intermediate points. These function are used for pure visualization, they are not related to the E-Nautilus procedure.

In [ ]:
def display_options(zs, z_scaler=None, nadir=None, ideal=None, is_max=None, best=None, names=None, titles=None, rows=2, cols=2):
    """Plots a radial scatter plot showing different solutions and reachable values. 
    
        Args:
            zs (np.ndarray): A 2D array with objective vectors to be displayed.
            z_scaler (Optional[sklearn.preprocessing.data.MinMaxScaler]): Fitted scaler used to transform
            the normalized data into it's original scale.
            nadir (Optional[np.ndarray]): A 1D array representing the nadir point.
            ideal (Optional[np.ndarray]): A 1D array representing the ideal point.
            is_max (Optional[List[bool]]): A 1D array with each truth value representing if an 
            objective funtion should be maximized or minimized. True implies maximization.
            best (Optional[np.ndarray]): A 2D array representing the best reachable values from zs
            (E-NAUTILUS specific)
            names (Optional[List[str]]): List of names for each of the objectives.
            rows (Optional[int]): How many subplots in each row should be displayed.
            cols (Optional[int]): How many subplots in each column should be displayed.
    """

    if z_scaler is not None:
        zs = z_scaler.inverse_transform(zs)
        if best is not None:
            best = z_scaler.inverse_transform(best)

    if is_max is None:
        # assume all to be minimized
        is_max = np.full(zs.shape[1], False)

    # Transform maximized objectives to an interpetable form
    zs_minmax = np.where(is_max, -zs, zs)
    if best is not None:
        best_minmax = np.where(is_max, -best, best)

    if nadir is not None and ideal is not None:
        if z_scaler is not None:
            nadir = z_scaler.inverse_transform(nadir.reshape(1, -1))
            ideal = z_scaler.inverse_transform(ideal.reshape(1, -1))
        up, lb = np.where(is_max, -ideal, nadir), np.where(is_max, -nadir, ideal)
    else:
        up, lb = (np.where(is_max, -np.max(zs, axis=0), np.min(zs, axis=0)),
                  np.where(is_max, -np.min(zs, axis=0), np.max(zs, axis=0)))

    # Scale the data between up and lb, normalizing it between 0 and 1 for all features
    # This is done because radial scatter plots don't support ranges for invidual radial axes.
    # Negative numbers are also not supported.
    if nadir is not None and ideal is not None:
        scaler = MinMaxScaler()
        scaler.fit(np.vstack((up, lb)))
    else:
        scaler = MinMaxScaler((0.1, 1))
        scaler.fit(np.vstack((up, lb)))

    # scaled solutions
    z_scaled = scaler.transform(zs_minmax)
    if best is not None:
        shadow = scaler.transform(best_minmax)
    
    # setup the figure
    if titles is None:
        titles = ["Candidate {}".format(i) for i in range(1, len(zs)+1)]
    fig = make_subplots(rows=rows,
                        cols=cols,
                        specs=[[{'type':'polar'}]*cols]*rows,
                        subplot_titles=titles,
                       )
    fig["layout"]["width"] = cols*500
    fig["layout"]["height"] = rows*500
    fig["layout"]["autosize"] = False
    polars = ["polar"] + ["polar{}".format(i+1) for i in range(1, len(zs))]

    dicts = dict(zip(polars,
                    [dict(radialaxis=dict(visible=False, range=[0, 1]))]*len(polars)))
    fig.update_layout(**dicts,
                     title=go.layout.Title(
                     text="Candidate solutions in Blue,\nbest reachable values in red.",
                     xref="container",
                     x=0.5,
                     xanchor="center",
                    ))
    
    if names is None:
        names = ["Objective {}".format(i) for i in range(1, zs.shape[1] + 1)]

    
    def index_generator():
        for i in range(1, rows+1):
            for j in range(1, cols+1):
                yield i, j
                
    gen = index_generator()
    traces = []
    
    for (z_i, z) in enumerate(zs):
        try:
            i, j = next(gen)
        except StopIteration:
            break
            
        if best is not None:
            fig.add_trace(
                go.Scatterpolar(
                    r=shadow[z_i],
                    opacity=1,
                    theta=names,
                    name="best",
                    fillcolor='red',
                    fill='toself',
                    showlegend=False,
                    hovertext=best_minmax[z_i],
                    hoverinfo='name+theta+text',                    
                    line={'color': 'red'},
                ),
                row=i,
                col=j,
            )

        fig.add_trace(
            go.Scatterpolar(
                r=z_scaled[z_i],
                opacity=0.5,
                theta=names,
                showlegend=False,
                name="Candidate {}".format(z_i+1),
                fill='toself',
                fillcolor='blue',
                hovertext=zs_minmax[z_i],
                hoverinfo='name+theta+text',
                line={'color': 'blue'},
            ),
            row=i,
            col=j,
        )
        
        if z_i == 0:
            polar = "polar"
        else:
            polar = "polar{}".format(z_i+1)

    for annotation in fig['layout']['annotations']: 
        annotation['yshift']= 20

    fig.show()
    
def z_selector(zs):
    res = widgets.Dropdown(
    options=[i for i in range(1, len(zs)+1)])
    return res

## Loading and scaling the data
Next we need to load the representation of the Pareto front into two arrays. One containing the solutions and the other containing the corresponding objective vectors.  Normalize the data between 0 and 1 column wise. The scalers are needed later to scale the data back into its' original scale for visualizations and to display the final results.

In [ ]:
decision_variables_file = "./data/decision_result.csv"
objective_vectors_file = "./data/objective_result.csv"
xs = np.genfromtxt(decision_variables_file, delimiter=',')
fs = np.genfromtxt(objective_vectors_file, delimiter=',')

# Defie which objectives are to be maximized
is_max = np.array([True, True, False, False, False])
fs = np.where(fs, -fs, fs)

scaler = MinMaxScaler()
scaler.fit(fs)
fs_norm = scaler.transform(fs)

## Using E-NAUTILUS
Create an ENautilus object and initialize it with a ScalarDataProblem containing variable values and objective vectors. We choose to iterate for 3 iterations and to generate 4 intermediate points during each iteration.

In [ ]:
problem = ScalarDataProblem(xs, fs_norm)
enautilus = ENautilus(problem)
total_iters = 3
points_shown = 4

nadir, ideal = enautilus.initialize(total_iters, points_shown)

After initializing the method, we get the nadir and ideal points estimated from the given objective vectors. Iterations starts from the nadir point. Notice how the nadir is all ones and the ideal is all zeros. This is due to the normalization. 

### NOTE ON THESE PLOTS:
The idea is, that when we are maximizing, the best possible value (ideal) is on the edge, and the worst (nadir) in the center. When minimizing, the best possible value (ideal) is in the center and the worst (nadir) on the edges. **The plot showing the nadir and ideal points, however, makes a small exeption. The values supposed to be in the center, are not exactly in the center, but close, as can be seen.** This is because otherwise, all the points would clump together and they could not be explored. In the later plots, the true ideal (if minimizing, otherwise nadir) point is dead in the center. 

In [ ]:
display_options(np.array([nadir, ideal]), z_scaler=scaler, titles=["Nadir", "Ideal"], is_max=is_max, rows=1, cols=2)

Run the first iteration which returns the first intermediate points and the corresponding best reachable objective vectors from each of these intermediate points. The best values are shown as the red areas, with values for each objective on the apexes. The value of the intermediate points as the blue areas with values on the apexes. Mouse over to explore.

In [ ]:
zs, best = enautilus.iterate()
display_options(zs, z_scaler=scaler, nadir=nadir, ideal=ideal, best=best, rows=4, cols=3, is_max=is_max)
s = z_selector(zs)
display(s)

We can specify our preferred point to the algorithm in the interaction phase from which the next intermediate points are to be generated. You can use the drop down menu above to select a preferred candidate.

In [ ]:
# s.value is just the value selected in the dropdown menu. Substract one from it for indexing to work.
left = enautilus.interact(zs[s.value-1], best[s.value-1])
print("Iterations left:", left)

Iterate again from the new selected point.

In [ ]:
zs, best = enautilus.iterate()
display_options(zs, z_scaler=scaler, nadir=nadir, ideal=ideal, best=best, rows=3, cols=2, is_max=is_max)
s = z_selector(zs)
display(s)

In [ ]:
left = enautilus.interact(zs[s.value-1], best[s.value-1])
print("Iterations left:", left)

In [ ]:
zs, lows = enautilus.iterate()
display_options(zs, z_scaler=scaler, nadir=nadir, ideal=ideal, best=None, rows=3, cols=2, is_max=is_max)
s = z_selector(zs)
display(s)

## Final solution
We ahve reached the final solution.

We call interact for the last time, and this time, it will yield the point selected in the last iteration. The objective values are scaled back to the original values.

(Later on, the option to project this last point to the Pareto front for example by minimizing an ASF with the last selected point being a reference point, will be added.)

In [ ]:
x, f = enautilus.interact(zs[s.value-1], best[s.value-1])
print("Final solution:", x)
print("With final objective values:", scaler.inverse_transform(f.reshape(1, -1)))